In [1]:
import os
from util import data
from time import time
from pdb import set_trace
from multiprocessing import Pool

%load_ext autoreload
%autoreload 2

default_pool_size = max(1, os.cpu_count())


music21: Certain music21 functions might need these optional packages: matplotlib, scipy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
scores = data.load_data()

Serialized scores found, loading...
Scores loaded in 24.69 seconds.


In [3]:
dataset = data.HaydnDataset(data=scores)
print("There are {} items in the dataset.".format(len(dataset)))

Building dataset...
Finished building dataset in 7.60 seconds.
There are 99 items in the dataset.


In [4]:
# get all of the datasets and filter out the empty ones
start = time()
states = list(filter(lambda ds: ds is not None, dataset))
total_ticks = sum(map(lambda state: state.shape[2], states)) * states[0].shape[0]
print("There are {} final corpi with a total of {} ticks. Loaded in {:.2f} seconds.".format(len(states), total_ticks, time()-start))

There are 81 final corpi with a total of 1872988 ticks. Loaded in 27.24 seconds.


In [18]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # useful stateless functions
import numpy as np
from torch.utils import data

In [19]:
class Center_Model(nn.Module):
    def __init__(self, center_input_size, center_output_size):
        super(Center_Model, self).__init__()
        
        self.center_affine = nn.Linear(center_input_size, center_output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, center_input):
        
        center_out = self.center_affine(center_input)
        output = self.relu(center_out)
        output = self.softmax(output)
        
        return output
  

In [20]:
class Result_Model(nn.Module):
    def __init__(self, merged_input_size, output_size):
        super(Result_Model, self).__init__()
        
        self.affine = nn.Linear(merged_input_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, merged_input):
        
        out = self.affine(merged_input)
        output = self.relu(out)
        output = self.softmax(output)
        
        return output

In [21]:
class Left_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Left_Model, self).__init__()
        
        self.left_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, left_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn output shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, output_size]
        '''
        
        left_out, (left_hn, left_cn) = self.left_lstm(left_seq)
        seq_len, batch, size = left_out.shape
        left_results  = left_out.view(seq_len * batch, size)
        
        out = self.affine(left_results)
        output = self.relu(out)
        output = self.softmax(output)
        
        return output, left_hn

In [22]:
class Right_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Right_Model, self).__init__()
        
        self.right_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, right_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn  shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, output_size]
        '''
        right_out, (right_hn, right_cn) = self.right_lstm(right_seq)
        
        seq_len, batch, size = right_out.shape
        right_results  = right_out.view(seq_len * batch, size)
        
        out = self.affine(right_results)
        output = self.relu(out)
        output = self.softmax(output)
        
        return output, right_hn

In [ ]:
class Arti_LSTM_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Left_Model, self).__init__()
        
        self.left_articulate = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, left_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn output shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, 1]
        '''
        
        left_out, (left_hn, left_cn) = self.left_articulate(left_seq)
        seq_len, batch, size = left_out.shape
        left_results  = left_out.view(seq_len * batch, size)
        
        out = self.affine(left_results)
        output = self.sigmoid(out)
        
        return output, left_hn

In [23]:
def train(left_model, right_model, center_model, result_model, arti_model
          left_tensor, right_tensor, center_tensor, 
          left_target_tensor, right_targte_tensor, center_target_tensor, arti_target_tensor
          opt_left, opt_right, opt_center, opt_affine, opt_arti):        
    
    left_output, left_hn = left_model(left_tensor,)
    right_output, right_hn = right_model(right_tensor)
    center_output = center_model(center_tensor)
    arti_output = arti_model(center_tensor)
    
    left_loss = F.nll_loss(left_output, left_target_tensor)
    opt_left.zero_grad()
    left_loss.backward(retain_graph=True)
    opt_left.step()
    
    right_loss = F.nll_loss(right_output, right_targte_tensor)
    opt_right.zero_grad()
    right_loss.backward(retain_graph=True)
    opt_right.step()
    
    center_loss = F.nll_loss(center_output, center_target_tensor)
    opt_center.zero_grad()
    center_loss.backward(retain_graph=True)
    opt_center.step()
    
    arti_loss = F.l1_loss()
    
    '''
    Get the last predicted output from LSTM and reshape to column vector
    '''
    left_out_seq_final = left_output[-1, :].view(1, -1)
    right_out_seq_final = right_output[-1, :].view(1, -1)
#     print('left_out_seq_final shape: {x}'.format(x=left_out_seq_final.shape))
#     print('center_output shape: {x}'.format(x=center_output.shape))
#     print('right_out_seq_final shape: {x}'.format(x=right_out_seq_final.shape))
    comobined_result = torch.cat((left_out_seq_final, center_output, right_out_seq_final), 1)
    result_output = result_model(comobined_result)
 
    result_loss = F.nll_loss(result_output, center_target_tensor)
    opt_affine.zero_grad()
    result_loss.backward(retain_graph=True)
    opt_affine.step()
        
    return result_output, left_loss.item(), right_loss.item(), center_loss.item(), result_loss.item()

In [24]:
def findNoneZeroIndex(m):
    '''
    m is a 2 dimentional numpy array
    
    m is modified that the last entry is deleted. 
    
    return an array with shape [k, ] contains the (only) nonzero index of each row in m
    k = the number of rows in m
    '''
    x, y = m.shape
#     print('m shape: {x}'.format(x=m.shape))
    n = np.zeros(x)
    for i in range (x):
        m_row = m[i, :]
        m_row_clipped = np.delete(m_row, -1)
#         print('m_row_clipped shape: {x}'.format(x=m_row_clipped.shape))
#         print(m_row_clipped)
        index = np.nonzero(m_row_clipped)[0]
#         print('idx: {x}'.format(x=index))
        n[i] = index
    return n

In [25]:
def getTargetIndexTensor(left, right, c_target_tensor):
    center_clip = np.delete(c_target_tensor, -1)
    center_target_index_array = np.nonzero(center_clip)[0]
    center_target_index_tensor = torch.from_numpy(center_target_index_array).long()

    
    left_target = findNoneZeroIndex(left)
    right_target = findNoneZeroIndex(right)
    right_index_target = np.flip(right_target, axis=0)
    
    left_index_target_tensor = torch.from_numpy(left_target).long()
    right_index_target_tensor = torch.from_numpy(right_index_target.copy()).long()
    
    return (left_index_target_tensor, right_index_target_tensor, center_target_index_tensor)

In [35]:
def generateMiniBatchFromOneMusicForPart(music_data, seq_length, part):
    '''    
    input: music_data. A numpy array with shape [4, tick, data_dim]
    return: a list contains miniBatch data for one part. Each item in the miniBatch list contains:
            - left_tensor: shape [seq_length, 1, D]
            - right_tensor: shape [seq_length, 1, D]
            - center_tensor: shape [1, D]
            - left_target_tensor: shape [seq_length, ]
            - right_targte_tensor: shape [seq_length, ]
            - center_target_tensor: shape [1, 1]
            - articulation_target_tensor: shape[1,]

        D = size of array contains the data from three other parts
                
    '''
    miniBatches = []
    
    _, music_length, data_dim = music_data.shape
#     music_length = 6 # this is only for testing
    i = 0 # TODO: maybe add functinality to pad the beginning and end.
    center_inx = i + seq_length
    while (center_inx + seq_length) < music_length:
        left_range_start = center_inx - seq_length
        left_range_end = center_inx 
        center_index = center_inx
        right_range_start = center_inx + 1
        right_range_end = right_range_start + seq_length
        
        left_range = music_data[part, left_range_start:left_range_end, :]
        seq_l, notes_l = left_range.shape
        left_tensor = torch.from_numpy(left_range.reshape(seq_l, 1, notes_l)).float()
        
        right_range = music_data[part, right_range_start:right_range_end, :]
        right_flipped = np.flip(right_range, axis=1).copy()
        seq_l, notes_l = right_flipped.shape
        right_tensor = torch.from_numpy(right_flipped.reshape(seq_l, 1, notes_l)).float()
        
        center_rep = music_data[:, center_inx, :]
        center_other_parts = np.delete(center_rep, part, axis=0)
        part_c, notes_c = center_other_parts.shape
        center_tensor = torch.from_numpy(center_other_parts.reshape(1, part_c * notes_c)).float()
        
        '''
        center_target_range shape: [data_dim,]
        '''
        center_target_range = music_data[part, center_inx+1, :]
        
        '''
        left range and right range are in dimension [seg_length, data_dim]
        '''
        left_target_tensor_range = music_data[part, left_range_start+1:left_range_end+1, :]
        right_target_tensor_range = music_data[part, right_range_start-1:right_range_end-1, :]

        '''
        Get target tensor for left, right, and center note model.
        '''
        left_index_target_tensor, right_index_target_tensor, center_index_target_tensor = getTargetIndexTensor(
            left_target_tensor_range, right_target_tensor_range, center_target_range)
        
        '''
        articulation target is just tensor that is a number
        '''
        last_index = data_dim - 1
        articulate_y = music_data[part, center_inx, last_index]
        articulate_target = torch.tensor(articulate_y).float()
        
        miniBatches.append((left_tensor, right_tensor, center_tensor, left_index_target_tensor, right_index_target_tensor, center_index_target_tensor, articulate_target))
        
        i += 1
        center_inx = i + seq_length
        
    return miniBatches
    

In [32]:
'''
mini generateMiniBatchFromOneMusicForPart test
'''
# fake_data = np.array([[
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 0, 1]],

#        [[1, 0, 0],
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [0, 0, 1],
#         [0, 1, 0]],

#        [[1, 0, 0],
#         [0, 1, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [1, 0, 0]],

#        [[0, 1, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 0, 1],
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0]]])
# # print('fake_data')
# # print(fake_data)

# r = generateMiniBatchFromOneMusicForPart(fake_data, 3, 0)
# lt, rt, ct, lt_t, rt_t, ct_t = r[0]

'\nmini generateMiniBatchFromOneMusicForPart test\n'

In [33]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [37]:
demo_piece_list = dataset.__getitem__(3) #numpy array
demo_piece = demo_piece_list[0, :, :, :]
print('demo_piece shape: {x}'.format(x=demo_piece.shape))
parts, ticks, data_dim = demo_piece.shape

'''
data_dim should be 73
'''
note_size = data_dim - 1

lstm_input_size = data_dim
lstm_hidden_size = 200
lstm_output_size = note_size
center_input_size = data_dim * 3
center_output_size = note_size

arti_lstm_hidden_size = 200
arti_output_size = 1
arti_center_input_size = 3

n_iters = 2000
print_every = 100
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters
learning_rate = 0.005

'''
(1) Data representation:
Each tick contains 73 entries.
0-70: notes one-hot representation
71: boolean rest representation
72: boolean articulation representation


(2) Model ensemble:
Four ensemble models. Each part has its own ensemble models.
Each ensemble model contains note prediction models + articulation prediction models

NOTE PREDICTION models:
left_LSTM. [vector input_size:  73 * seq_length. expected vector output_size: 72]
right_LSTM. [vector input_size:  73 * seq_length. expected vector output_size: 72]
harmony_Linear (center_linear) [vector input_size:  73 * 3. expected vector output_size: 72]
result_Linear [vector input_size:  72 * 3. expected vector output_size: 72]

ARTICULATION PREDICTION models:
left_LSTM. [vector input_size:  73. expected vector output_size: 1]
right_LSTM. [vector input_size:  73. expected vector output_size: 1]
result_Linear. [vector input_size:  2. expected vector output_size: 1]

The final prediction for each tick is the combination of the note prediction and the articulation prediction.
Final vector output_size: 72 + 1 = 73

'''

left_model = Left_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
right_model = Right_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
center_model = Center_Model(center_input_size, center_output_size)
result_model = Result_Model(center_output_size + lstm_output_size * 2, note_size)
arti_left_model = Arti_LSTM_Model(lstm_input_size, arti_lstm_hidden_size, arti_output_size)

left_opt = optim.SGD(left_model.parameters(), lr=learning_rate)
right_opt = optim.SGD(right_model.parameters(), lr=learning_rate)
center_opt = optim.SGD(center_model.parameters(), lr=learning_rate)
result_opt = optim.SGD(result_model.parameters(), lr=learning_rate)
arti_left_opt = optim.SGD(result_model.parameters(), lr=learning_rate)

seq_length = 2

demo_piece_minibatch_pt1 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 0)
'''
TODO: put these lines back!
'''
# demo_piece_minibatch_pt2 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 1)
# demo_piece_minibatch_pt3 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 2)
# demo_piece_minibatch_pt4 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 3)
miniBatch_length = len(demo_piece_minibatch_pt1)
print("minibatch list length: {x}".format(x=miniBatch_length))

#Training for Part 1
start = time.time()
ite = 0
'''
IMPORTANT: Model, Optimizer must match!
'''

'''
NOTE: This is only trainning for part 1, with data from one music piece.

TODO:
(1) generate minibatch from other pieces
(2) write functions to generate minibatch contains N sequences. right now it's 1 sequence
(3) write sampling function
(4) fix other TODOs
(5) Add error catching blocks to skip ticks cause problems
(6) Tune hyperparameters: seq_length, learning rate (probably different for four models)
(7) train for other parts


'''
for n in range(n_iters):
    if ite/miniBatch_length >= 1:
        ite = 0
        
    '''
    TODO:
    Update this part to train 4 parts in one iteration.
    '''
    left_tensor, right_tensor, center_tensor, 
    left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target = demo_piece_minibatch_pt1[ite]

    output, left_loss, right_loss, center_loss, result_loss, arti_loss = train(left_model, right_model, center_model, result_model,arti_left_model 
          left_tensor, right_tensor, center_tensor, 
          left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target
          left_opt, right_opt, center_opt, result_opt, arti_left_opt)
    total_loss += result_loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) loss: %.4f' % (timeSince(start), n, n / n_iters * 100, result_loss))
            print("output: {x}".format(x=output))
            
    if (n+1) % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0
    
    ite += 1
    n += 1
    
# print(n)


demo_piece shape: (4, 1541, 73)
minibatch list length: 1537


/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


0m 0s (99 4%) loss: -0.0141
output: tensor([[0.0137, 0.0147, 0.0137, 0.0146, 0.0145, 0.0139, 0.0137, 0.0140, 0.0141,
         0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0139, 0.0139, 0.0144, 0.0137,
         0.0145, 0.0137, 0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0140, 0.0137,
         0.0139, 0.0137, 0.0141, 0.0137, 0.0146, 0.0142, 0.0137, 0.0137, 0.0137,
         0.0137, 0.0137, 0.0137, 0.0137, 0.0145, 0.0137, 0.0138, 0.0137, 0.0137,
         0.0137, 0.0137, 0.0146, 0.0137, 0.0144, 0.0140, 0.0137, 0.0142, 0.0137,
         0.0144, 0.0138, 0.0145, 0.0140, 0.0137, 0.0143, 0.0137, 0.0137, 0.0137,
         0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0142, 0.0137, 0.0137, 0.0137]],
       grad_fn=<SoftmaxBackward>)
0m 1s (199 9%) loss: -0.0137
output: tensor([[0.0137, 0.0147, 0.0137, 0.0146, 0.0145, 0.0139, 0.0137, 0.0139, 0.0140,
         0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0139, 0.0139, 0.0144, 0.0137,
         0.0145, 0.0137, 0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0140, 0.0137,


0m 7s (1299 64%) loss: -0.0137
output: tensor([[0.0137, 0.0147, 0.0137, 0.0146, 0.0145, 0.0139, 0.0137, 0.0139, 0.0140,
         0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0139, 0.0139, 0.0144, 0.0137,
         0.0145, 0.0137, 0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0140, 0.0137,
         0.0139, 0.0137, 0.0141, 0.0137, 0.0146, 0.0143, 0.0137, 0.0137, 0.0138,
         0.0137, 0.0137, 0.0137, 0.0137, 0.0145, 0.0137, 0.0138, 0.0137, 0.0137,
         0.0137, 0.0137, 0.0145, 0.0137, 0.0144, 0.0140, 0.0137, 0.0142, 0.0137,
         0.0144, 0.0138, 0.0144, 0.0140, 0.0137, 0.0143, 0.0137, 0.0137, 0.0137,
         0.0137, 0.0137, 0.0137, 0.0137, 0.0137, 0.0142, 0.0137, 0.0137, 0.0137]],
       grad_fn=<SoftmaxBackward>)
0m 7s (1399 69%) loss: -0.0137
output: tensor([[0.0137, 0.0147, 0.0137, 0.0146, 0.0144, 0.0139, 0.0137, 0.0139, 0.0140,
         0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0139, 0.0139, 0.0144, 0.0137,
         0.0145, 0.0137, 0.0137, 0.0137, 0.0137, 0.0143, 0.0137, 0.0140, 0.0

In [ ]:
print(demo_piece[0][10])